# tfds Dataset Creation

In [ ]:
from google.colab import drive
#drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import os

In [ ]:
os.getcwd()

'/content'

In [ ]:
!tar -xvf '/content/drive/MyDrive/State_Train.tar' 

In [ ]:
!pip uninstall tensorflow_datasets

In [ ]:
!pip install tensorflow_datasets

In [ ]:
!nvidia-smi

In [ ]:
!tfds new state_dataset

In [ ]:
!tfds build state_dataset

# Mode Building and Training


In [ ]:
import os
import numpy as np
from random import shuffle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Activation,Dropout,BatchNormalization,Input,MaxPooling2D
import tensorflow_datasets as tfds

In [ ]:
STATE_NAMES = {0: 'USA_RI', 1: 'USA_DC', 2: 'USA_VT', 3: 'USA_PR', 4: 'USA_WY', 5: 'USA_NV', 6: 'USA_UT', 7: 'USA_WA', 8: 'USA_OR',
                9: 'USA_NJ', 10: 'USA_HI', 11: 'USA_ND', 12: 'USA_KY', 13: 'USA_AR', 14: 'USA_AZ', 15: 'USA_CA', 16: 'USA_CO', 17: 'USA_FL',
                18: 'USA_GA', 19: 'USA_IA', 20: 'USA_IL', 21: 'USA_IN', 22: 'USA_KS', 23: 'USA_LA', 24: 'USA_ME', 25: 'USA_MI', 26: 'USA_MN',
                27: 'USA_MO', 28: 'USA_MS', 29: 'USA_NC', 30: 'USA_NM', 31: 'USA_OH', 32: 'USA_OK', 33: 'USA_TN', 34: 'USA_WI', 35: 'USA_ID',
                36: 'USA_PA', 37: 'USA_SC', 38: 'USA_NH', 39: 'USA_VA', 40: 'USA_NY', 41: 'USA_CT', 42: 'USA_DE', 43: 'USA_MD', 44: 'USA_AK',
                45: 'USA_MT', 46: 'USA_GOV', 47: 'USA_NE', 48: 'USA_AL', 49: 'USA_WV', 50: 'USA_SD', 51: 'USA_TX_TEMP', 52: 'USA_TX_1', 53: 'USA_TX_2'}

ds,ds_info = tfds.load('state_dataset',split=['train[:85%]','train[85%:]'],with_info=True)

train= ds[0]
valid= ds[1]

IMAGE_SIZE = (32,128)
BATCH_SIZE = 64

def img_resize(x):
    img = tf.image.resize(x['image'],IMAGE_SIZE)
    label = tf.one_hot(x['label'],54)
    return img, label

train_data = (train.map(img_resize)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
valid_data = (valid.map(img_resize)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
EPOCHS = 50

model =tf.keras.Sequential()
model.add(Conv2D(filters=64,padding='same',kernel_size=3,activation='relu',input_shape=(32,128,3)))
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

model.add(tf.keras.layers.Conv2D(filters=128,padding='same',kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

model.add(tf.keras.layers.Conv2D(filters=256,padding='same',kernel_size=3,activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=256,padding='same',kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,1)))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(54,activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

filepath = "/content/drive/MyDrive/State_model_with_weights/State_classificationModel_tfds_for_50_test_colab.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(logs_dir='logs',filepath=filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto',save_weights_only=False)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10),
tensor_board_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs')
callbacks_list = [early_stopping,checkpoint,tensor_board_callback]

history = model.fit(train_data,epochs=EPOCHS,callbacks=callbacks_list,verbose=1, validation_data=valid_data)


Epoch 1/50
21481/21481 [==============================] - ETA: 0s - loss: 0.4792 - accuracy: 0.9011
Epoch 1: val_accuracy improved from -inf to 0.91852, saving model to /content/drive/MyDrive/State_model_with_weights/State_classificationModel_tfds_for_50_test_colab.hdf5
21481/21481 [==============================] - 349s 16ms/step - loss: 0.4792 - accuracy: 0.9011 - val_loss: 0.3676 - val_accuracy: 0.9185
Epoch 2/50
21477/21481 [============================>.] - ETA: 0s - loss: 0.3104 - accuracy: 0.9349
Epoch 2: val_accuracy improved from 0.91852 to 0.93802, saving model to /content/drive/MyDrive/State_model_with_weights/State_classificationModel_tfds_for_50_test_colab.hdf5
21481/21481 [==============================] - 336s 16ms/step - loss: 0.3103 - accuracy: 0.9349 - val_loss: 0.2994 - val_accuracy: 0.9380
Epoch 3/50
21481/21481 [==============================] - ETA: 0s - loss: 0.2831 - accuracy: 0.9406
Epoch 3: val_accuracy did not improve from 0.93802
21481/21481 [===============

In [ ]:
## For Inference
pred_model = tf.keras.models.load_model('/content/drive/MyDrive/State_model_with_weights/State_classificationModel_tfds_for_50_test_colab.hdf5')

In [ ]:
path ="/content/PGBTMLP6-0086185E9AB6ADB048C0EE3.JPG"
image = tf.io.read_file(path)
image = tf.image.decode_image(image,channels=3)
image = tf.image.resize(image,(32,128))
image = tf.expand_dims(image,axis=0)
predictions = pred_model.predict(image)

class_index = np.argmax(predictions)
state = STATE_NAMES[class_index]
confidence_score = predictions[0][np.argmax(predictions)]

In [ ]:
print(state)
print(confidence_score)

USA_TX_2
0.99998784


In [ ]:
!tar -xvf ROIs.tar

In [ ]:
from tqdm import tqdm

In [ ]:
rois_path = '/content/ROIs'
with open('/content/drive/MyDrive/ROIs_state.txt', 'a+') as f:
  for file in tqdm(os.listdir(rois_path)):
    #print(file)
    image = tf.io.read_file(os.path.join(rois_path,file))
    if(len(image.numpy())!=0):
      image = tf.image.decode_image(image,channels=3)
      image = tf.image.resize(image,(32,128))
      image = tf.expand_dims(image,axis=0)
      predictions = pred_model.predict(image)
      class_index = np.argmax(predictions)
      state = STATE_NAMES[class_index]
      confidence_score = predictions[0][np.argmax(predictions)]
      f.write(file+","+state+","+str(confidence_score)+"\n")
    else:
      f.write(file+","+" could not read image"+"\n")


100%|██████████| 8416/8416 [05:46<00:00, 24.29it/s]


In [ ]:
im = tf.io.read_file(os.path.join(rois_path,'0092424100220210413170206530000_2.JPG'))

In [ ]:
len(im.numpy())

3899